In [1]:
#imports
import pandas as pd
import os, ipdb, re
import random, evaluate
import string
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_dataset
import wandb
import ast
import re, os
import subprocess

2023-12-08 09:19:44.817078: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-08 09:19:44.868569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 09:19:46.137547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
i = 1
# dataset_path = f"../data/LLLM_DOCTEAT_TDMS_SQUAD_{i}/fold1"
# dataset_path = f"../data/LLLM_LONG_TDMS_SQUAD_{i}/fold1"
dataset_path = f"../data/LLLM_LONG_SUMMARIZED_TDMS_SQUAD_{i}/fold1"


# dataset_path = f"../data/LLLM_LONG_TDMS_DROP_{i}/fold2"
# dataset_path = "../data/LLLM_DOCTEAT_TDMS_ALL_TEMPLATE/fold1"
# dataset_path = f"../data/LLLM_LONG_TDMS_SQUAD_{i}/fold1"
# dataset_path = "../data/LLLM_LONG_TDMS_ALL_TEMPLATE/fold1"

dataset_dict = DatasetDict.load_from_disk(f"{dataset_path}")
    

train_data = dataset_dict["train"].shuffle(seed=42)
valid_data = dataset_dict["validation"].shuffle(seed=42)
# valid_data = dataset["validation"].shard(num_shards=10, index=0).shuffle(seed=42)

# train_data[0]
len(valid_data)

1298

In [3]:
train_data[1704]

{'prompt': 'Title:\tCompatibility of Shelah and Stupp’s and Muchnik’s iteration with fragments of monadic second order logic\n\nAbstract:\tWe investigate the relation between the theory of the iterations in the sense of Shelah-Stupp and of Muchnik, resp., and the theory of the base structure for several logics. These logics are obtained from the restriction of set quantification in monadic second order logic to certain subsets like, e.g., finite sets, chains, and finite unions of chains. We show that these theories of the Shelah-Stupp iteration can be reduced to corresponding theories of the base structure. This fails for Muchnik’s iteration.\n\nDietrich Kuske\n\n\nPlease answer a question about this article. If the question is unanswerable, say "unanswerable". What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, metric, and score?',
 'answer': 'unanswerable',
 '__index_level_0__': 2695}

In [4]:
len(train_data[1704]['prompt'].split("\n\n"))

4

In [5]:
# train_data[1704]['prompt'].split("\n\n")

In [6]:
valid_data[random.randint(0, len(valid_data))]

# valid_data[1295]


{'prompt': 'Title:\tLinguistic Input Features Improve Neural Machine Translation\n\nAbstract:\tNeural machine translation has recently achieved impressive results, while using little in the way of external linguistic information. In this paper we show that the strong learning capability of neural MT models does not make linguistic features redundant; they can be easily incorporated to provide further improvements in performance. We generalize the embedding layer of the encoder in the attentional encoder–decoder architecture to support the inclusion of arbitrary features, in addition to the baseline word feature. We add morphological features, part-of-speech tags, and syntactic dependency labels as input features to English↔German and English→Romanian neural machine translation systems. In experiments on WMT16 training and test sets, we find that linguistic input features improve model quality according to three metrics: perplexity, Bleu and chrF3. An open-source implementation of our n

In [7]:
valid_data[10]['prompt']

'Title:\tColorNet - Estimating Colorfulness in Natural Images\n\nAbstract:\tMeasuring the colorfulness of a natural or virtual scene is critical for many applications in image processing field ranging from capturing to display. In this paper, we propose the first deep learning-based colorfulness estimation metric. For this purpose, we develop a color rating model which simultaneously learns to extracts the pertinent characteristic color features and the mapping from feature space to the ideal colorfulness scores for a variety of natural colored images. Additionally, we propose to overcome the lack of adequate annotated dataset problem by combining/aligning two publicly available colorfulness databases using the results of a new subjective test which employs a common subset of both databases. Using the obtained subjectively annotated dataset with 180 colored images, we finally demonstrate the efficacy of our proposed model over the traditional methods, both quantitatively and qualitativ

In [8]:
print(re.search(f"Data-to-Text Generation", valid_data[10]['prompt']))

None


In [9]:
from fuzzywuzzy import fuzz
similarity = fuzz.ratio("Data-to-Text Generation", "Text Generation")
similarity

/nfs/home/kabenamualus/.local/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


79

In [10]:
dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}

print(dfs['validation'].head())

                                              prompt  \
0  Title:\t Dark Model Adaptation: Semantic Image...   
1  Title:\tGeometry Normalization Networks for Ac...   
2  Title:\tA Better Baseline for AVA\n\nAbstract:...   
3  Title:\tContextNet: Exploring Context and Deta...   
4  Title:\tGlossBERT: BERT for Word Sense Disambi...   

                                              answer  __index_level_0__  
0  [{'LEADERBOARD': {'Task': 'Semantic Segmentati...                  0  
1  [{'LEADERBOARD': {'Task': 'Scene Text Detectio...                  1  
2  [{'LEADERBOARD': {'Task': 'Action Recognition'...                  2  
3  [{'LEADERBOARD': {'Task': 'Semantic Segmentati...                  3  
4  [{'LEADERBOARD': {'Task': 'Word Sense Disambig...                  4  


In [11]:
dfs['validation'].head()

,prompt,answer,__index_level_0__
0,Title:\t Dark Model Adaptation: Semantic Image...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...,0
1,Title:\tGeometry Normalization Networks for Ac...,[{'LEADERBOARD': {'Task': 'Scene Text Detectio...,1
2,Title:\tA Better Baseline for AVA\n\nAbstract:...,[{'LEADERBOARD': {'Task': 'Action Recognition'...,2
3,Title:\tContextNet: Exploring Context and Deta...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...,3
4,Title:\tGlossBERT: BERT for Word Sense Disambi...,[{'LEADERBOARD': {'Task': 'Word Sense Disambig...,4


In [15]:
dfs['train']["prompt_lenght"] = dfs['train']["prompt"].apply(lambda x: len(x.split()))
dfs['train']["answer_lenght"] = dfs['train']["answer"].apply(lambda x: len(x.split()))

dfs['validation']["prompt_lenght"] = dfs['validation']["prompt"].apply(lambda x: len(x.split()))
dfs['validation']["answer_lenght"] = dfs['validation']["answer"].apply(lambda x: len(x.split()))

In [16]:
dfs['train'].describe()

,__index_level_0__,prompt_lenght,answer_lenght
count,3075.000000,3075.000000,3075.000000
mean,1537.000000,1670.739187,55.449756
std,887.820365,1149.200513,104.680918
min,0.000000,48.000000,1.000000
25%,768.500000,785.500000,11.000000
50%,1537.000000,1682.000000,27.000000
75%,2305.500000,2289.000000,64.000000
max,3074.000000,16491.000000,2561.000000


In [17]:
dfs['validation'].describe()

,__index_level_0__,prompt_lenght,answer_lenght
count,1298.000000,1298.000000,1298.000000
mean,648.500000,1642.204931,54.721109
std,374.844634,1097.751424,94.978970
min,0.000000,49.000000,1.000000
25%,324.250000,801.000000,11.000000
50%,648.500000,1656.000000,26.000000
75%,972.750000,2249.750000,64.000000
max,1297.000000,10590.000000,1870.000000


In [32]:
2353*15

35295

In [33]:
1876+2946

4822

In [34]:
1876+3753

5629

In [35]:
5512 - 3753

1759

In [36]:
2353 - 1608

745

# Stats

In [37]:
import numpy as np

def describe_list(lst):
    # Count
    count = len(lst)
    
    # Mean
    mean = np.mean(lst)
    
    # Standard deviation
    std = np.std(lst)
    
    # Minimum
    min_val = np.min(lst)
    
    # 25th Percentile
    percentile_25 = np.percentile(lst, 25)
    
    # Median
    median = np.median(lst)
    
    # 75th Percentile
    percentile_75 = np.percentile(lst, 75)
    
    # Maximum
    max_val = np.max(lst)
    
    result = {
        'count': count,
        'mean': mean,
        'std': std,
        'min': min_val,
        '25%': percentile_25,
        '50%': median,
        '75%': percentile_75,
        'max': max_val
    }
    
    return result


def get_df_stats(df): 
    
    unique_labels = df.answer.tolist()
    
    TDMS = set()
    Uniq_task = set()
    Uniq_dataset = set()
    Uniq_metric = set()
    Uniq_score = set()
    unanswerable_count = 0
    tdms_per_paper = []
    
    for contrib in unique_labels:
        try:
            parsed_json = ast.literal_eval(contrib)
        except :
            # TODO: We need a best way to deal with this 

            if contrib == 'unanswerable':
                unanswerable_count += 1
            else:
                ipdb.set_trace()
                
                print(f"Error parsing: \n{contrib}")
                # missed += 1
                parsed_json =  contrib
        tdms_count = 0
        for leaderboard in parsed_json:

            task = leaderboard['LEADERBOARD']['Task'].strip()
            dataset = leaderboard['LEADERBOARD']['Dataset'].strip()
            metric = leaderboard['LEADERBOARD']['Metric'].strip()
            score = leaderboard['LEADERBOARD']['Score'].strip()

            TDMS.add(f"{task}#{dataset}#{metric}#{score}")
            
            Uniq_task.add(task)
            Uniq_dataset.add(dataset)
            Uniq_metric.add(metric)
            Uniq_score.add(score)
            tdms_count += 1
            # ipdb.set_trace()
        
        tdms_per_paper.append(tdms_count)
        
        
    print(f"Number of papers: {len(unique_labels)}")
    print(f"Unanswerable count: {unanswerable_count}")
    # print(f"Total leaderboards: {len(path_to_df[(path_to_df.label == True) & (path_to_df.TDM != 'unknown')].title.tolist())}")
    print(f"Avg leaderboard per paper: {round(np.mean(list(tdms_per_paper)), 2)}")
    print(f"Distinc leaderboard: {len(TDMS)}")
    print(f"Distinct taks: {len(Uniq_task)}")
    print(f"Distinc datasets: {len(Uniq_dataset)}")
    print(f"Distinc metrics: {len(Uniq_metric)}")
    print(f"Distinc score: {len(Uniq_score)}")
    print(f"Max leaderboard per paper: {round(np.max(list(tdms_per_paper)), 2)}")
    print(f"Min leaderboard per paper: {round(np.min(list(tdms_per_paper)), 2)}")

    describe_result = describe_list(tdms_per_paper)

    print(f"\nResults describe TDMS:")
    for key, value in describe_result.items():
        print(f"{key}: {value}")
    
    return tdms_per_paper

In [38]:
dfs['validation'].head()

,prompt,answer,__index_level_0__,prompt_lenght,answer_lenght
0,Dark Model Adaptation: Semantic Image Segmenta...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...,0,353,11
1,Geometry Normalization Networks for Accurate S...,[{'LEADERBOARD': {'Task': 'Scene Text Detectio...,1,441,75
2,A Better Baseline for AVA We introduce a simpl...,[{'LEADERBOARD': {'Task': 'Action Recognition'...,2,526,24
3,ContextNet: Exploring Context and Detail for S...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...,3,424,11
4,Efficient Piecewise Training of Deep Structure...,[{'LEADERBOARD': {'Task': 'Semantic Segmentati...,4,406,24


In [39]:
tdms_per_paper_train = get_df_stats(dfs['train'])

Number of papers: 5512
Unanswerable count: 1872
Avg leaderboard per paper: 4.15
Distinc leaderboard: 20102
Distinct taks: 699
Distinc datasets: 2551
Distinc metrics: 1280
Distinc score: 9833
Max leaderboard per paper: 254
Min leaderboard per paper: 1

Results describe TDMS:
count: 5512
mean: 4.145500725689405
std: 8.27826236580898
min: 1
25%: 1.0
50%: 2.0
75%: 4.0
max: 254


In [40]:
tdms_per_paper_validation = get_df_stats(dfs['validation'])

Number of papers: 2353
Unanswerable count: 804
Avg leaderboard per paper: 15.4
Distinc leaderboard: 8691
Distinct taks: 497
Distinc datasets: 1639
Distinc metrics: 886
Distinc score: 5116
Max leaderboard per paper: 170
Min leaderboard per paper: 1

Results describe TDMS:
count: 2353
mean: 15.399915002124947
std: 15.016051476594187
min: 1
25%: 2.0
50%: 7.0
75%: 34.0
max: 170


In [84]:
round((4.15+4.52)/2, 2)

4.33

In [85]:
round((15.4+4.04)/2, 2)

9.72

In [ ]:
round(np.max(list(tdms_per_paper)), 2)

## Latex

In [3]:
!pwd

/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/notebooks


In [3]:
id_ = ["2105.02723v1","2104.06378v1","2104.04946v1","1912.03330v1", "1912.02738v4","1912.01326v3", "1912.00998v2"]
latex = f"/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/{id_[0]}.tex"
# latex = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/1312.6114v10.tex"

In [8]:

def remove_sections(latex_file):
    
    if len(latex_file.rsplit("/",1)) != 2:
        return 
    
    base_source, filename = latex_file.rsplit("/",1)

    if len(filename.rsplit(".",1)) != 2:
        return 
    
    file_id, file_ext = filename.rsplit(".",1)
    
    
    # Read the input LaTeX file
    with open(latex_file, 'r', encoding='utf-8') as file:
        content = file.read()

    # # Define the pattern to match sections to be removed
    # # This pattern looks for the string \section{Introduction} or \section{Experiment setup}
    # # followed by any content until the next occurrence of \section or the end of the file
    # pattern = re.compile(
    #     r'\\section\{Introduction\}.*?(?=\\section|\Z)|'
    #     r'\\section\{Related work\}.*?(?=\\section|\Z)|'
    #     r'\\section\{Experiment setup\}.*?(?=\\section|\Z)',
    #     re.DOTALL
    # )

    # Define the pattern to match sections to be removed
    # This pattern looks for the string \section, followed by any number of characters,
    # followed by either 'Introduction' or 'Experiment setup', followed by any characters
    # until the next occurrence of \section or the end of the file.
    # \s* matches any whitespace characters, and [^}]* matches any character except '}'
    pattern = re.compile(
        r'\\section\*?\s*\{[^}]*\b(Introduction(s?)|Related work(s?)|Future work(s?)|Background(s?)|Discussion(s?)|Methodology|Appendix|Supplementary|Supplemental)\b[^}]*\}.*?(?=\\section|\\end\{document\}|\\bibliography|\Z)',
        # r'\\section\s*\{[^}]*\b(Introduction|Related work|Future work)\b[^}]*\}.*?(?=\\section|\Z)',
        re.DOTALL | re.IGNORECASE
    )
    

    # pattern = re.compile(
    #     r'''
    #     (                             # Start capturing group
    #         \\section                 # Match \section
    #         \*?                       # Match optional *
    #         \s*                       # Match optional whitespace
    #         \{                        # Match {
    #         (?!                       # Start negative lookahead
    #             Result(s?)               # Negative lookahead for Results
    #             |                     # Or
    #             Experimentation(s?)       # Negative lookahead for Experimentation
    #             |
    #             Experiment(s?)
    #             |
    #             Conclusion
    #         )                         # End negative lookahead
    #         [^}]*                     # Match any characters except }
    #         \}                        # Match }
    #         .*?                       # Match any characters (non-greedy)
    #         (?=\\section|\Z)          # Positive lookahead for next \section or end of string
    #     )                             # End capturing group
    #     ''',
    #     re.DOTALL | re.IGNORECASE | re.VERBOSE
    # )



    # Remove the matched content
    content_new = re.sub(pattern, '', content)

    if not os.path.exists(f"{base_source}/edits"):
        os.makedirs(f"{base_source}/edits")
        
    if os.path.exists(f"{base_source}/edits/{file_id}_edit.{file_ext}"):
        os.remove(f"{base_source}/edits/{file_id}_edit.{file_ext}")
            
    # Write the modified content back to the file
    with open(f"{base_source}/edits/{file_id}_edit.{file_ext}", 'w', encoding='utf-8') as file:
        file.write(content_new)

# Replace 'yourfile.tex' with the path to your LaTeX file


In [9]:
source_folder = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/sample"
for file_id in os.listdir(f"{source_folder}"):
    latex_file = f"{source_folder}/{file_id}"
    remove_sections(latex_file)

In [2]:
id_ = ["2105.02723v1","2104.06378v1","2104.04946v1","1912.03330v1", "1912.02738v4","1912.01326v3", "1912.00998v2"]
latex_file = f"/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/{id_[0]}.tex"
# latex = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/1312.6114v10.tex"

def compile_latex(latex_file):
    # Compile the LaTeX file using pdflatex or any other LaTeX compiler
    subprocess.run(["pdflatex", latex_file])


In [5]:

# Example usage
# latex_file = 'your_latex_file.tex'


compile_latex(latex_file)

FileNotFoundError: [Errno 2] No such file or directory: 'pdflatex': 'pdflatex'